In [45]:
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import logging
import sys
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s',
    level=logging.DEBUG,
    filename='census-nb.log'
)

In [46]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
%autoreload 2
import uscensus
from uscensus.util.sqlalchemycache import SqlAlchemyCache

In [48]:
import sqlite3

In [49]:
from configparser import RawConfigParser

In [50]:
cp = RawConfigParser()

In [51]:
cp.read('C:/cygwin64/home/nkrishna/.census')

['C:/cygwin64/home/nkrishna/.census']

In [52]:
# Work around for some Windows TLS issues

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.poolmanager import PoolManager
import ssl

class MyAdapter(HTTPAdapter):
    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = PoolManager(num_pools=connections,
                                       maxsize=maxsize,
                                       block=block,
                                       ssl_version=ssl.PROTOCOL_TLSv1_2)
s = requests.Session()
s.mount('https://', MyAdapter())

In [53]:
%%prun
with uscensus.DBAPICache(sqlite3, database='uscensus.db') as cache:
    cl = uscensus.DiscoveryInterface(
        cp.get('census','api_key'),
        cache,
        session=s,
        index=True,
        vintage=2015
    )

Indexing metadata
Done indexing metadata
 

In [54]:
cl.search('American Community Survey')

[2015 American Community Survey 1-Year Estimates,
 2011-2015 American Community Survey Comparison Profile,
 2015 American Community Survey Subject Tables,
 2015 American Community Survey Subject Tables,
 2011-2015 American Community Survey 5-Year Profiles,
 2015 American Community Survey 1-Year Profiles,
 2011-2015 American Community Survey 5-Year Estimates,
 2015 American Community Survey 1-Year Data Profile 115th Congressional District,
 2015 American Community Survey Supplemental Estimates,
 2015 American Community Survey Comparison Profile]

In [55]:
results = _
acs1 = results[0]

In [33]:
acs1.searchVariables('education')

[]

In [97]:
cl.search('concepts:Income')

[2015 American Community Survey 1-Year Estimates,
 ACS 1-Year Detailed Tables,
 2011-2015 American Community Survey 5-Year Estimates,
 2015 American Community Survey Supplemental Estimates,
 2015 American Community Survey Subject Tables,
 2015 American Community Survey Subject Tables]

In [ ]:
api=cl.search('title:2015 American Community Survey Supplemental Estimates')[0]

In [99]:
api.variables

{'K201802_003MA': {'concept': 'K201802. Work Experience by Disability Status',
  'group': 'N/A',
  'label': 'Margin of Error for!!Worked full-time, year round:!!With a disability',
  'limit': 0,
  'predicateType': 'string',
  'validValues': []},
 'K201701_005EA': {'concept': 'K201701. Poverty Status in the Past 12 Months by Age',
  'group': 'N/A',
  'label': 'Income in the past 12 months below poverty level:!!65 years and over',
  'limit': 0,
  'predicateType': 'string',
  'validValues': []},
 'K201001_006E': {'concept': 'K201001. Marital Status for the Population 15 Years and Over',
  'group': 'N/A',
  'label': 'Divorced',
  'limit': 0,
  'predicateType': 'int',
  'validValues': []},
 'K200201_001E': {'concept': 'K200201. Race',
  'group': 'N/A',
  'label': 'Total:',
  'limit': 0,
  'predicateType': 'int',
  'validValues': []},
 'K202509_007M': {'concept': 'K202509. Housing Value',
  'group': 'N/A',
  'label': 'Margin of Error for!!$300,000 to $499,999',
  'limit': 0,
  'predicateType

In [101]:
api.variables['K202511_001E']

{'concept': 'K202511. Median Gross Rent (Dollars)',
 'group': 'N/A',
 'label': 'Median gross rent',
 'limit': 0,
 'predicateType': 'int',
 'validValues': []}

In [102]:
med_rent_by_county = api(['K202511_001E'], geo_for={'county': '*'})
med_rent_by_county.set_index(['state', 'county'], inplace=True)
med_rent_by_county.sort_index(inplace=True)
med_rent_by_county

K202511_001E
state county              
01    001            813.0
      003            914.0
      005            527.0
      007            697.0
      009            627.0
      013            567.0
      015            711.0
      017            680.0
      019            567.0
      021            723.0
      025            467.0
      031            658.0
      033            638.0
      039            602.0
      043            621.0
      045            650.0
      047            542.0
      049            645.0
      051            815.0
      053            548.0
      055            650.0
      059            529.0
      061            509.0
      069            713.0
      071            612.0
      073            823.0
      077            644.0
      079            563.0
      081            840.0
      083            605.0
...                    ...
72    081            340.0
      085            509.0
      087            317.0
      091            526.0
      097            353.0
      099            484.0
      101            485.0
      103            435.0
      105            395.0
      107            361.0
      111            452.0
      113            322.0
      115            447.0
      119            435.0
      121            528.0
      123            311.0
      125            349.0
      127            409.0
      129            408.0
      131            369.0
      133            300.0
      135            617.0
      137            490.0
      139            456.0
      141            240.0
      143            422.0
      145            563.0
      149            342.0
      151            402.0
      153             99.0

[1901 rows x 1 columns]

In [103]:
api.vintage

2015

In [104]:
%matplotlib inline

In [105]:
import matplotlib.pyplot as plt
import fiona
import geopandas as gpd
import shapely
import qgrid

In [106]:
%%time
# Generated in GetCountyShapes.ipynb
counties_df = gpd.GeoDataFrame.from_file('counties.geojson',
                                         driver='GeoJSON')
counties_df.crs = fiona.crs.from_epsg(4326)

Wall time: 55.5 s


In [ ]:
%time
counties_df.set_index(['STATE', 'COUNTY'], inplace=True)
counties_df.sort_index(inplace=True)

In [ ]:
%time
counties_df.index.names=['state','county']
joined = counties_df.join(med_rent_by_county)

In [ ]:
#joined.total_bounds
continental = shapely.geometry.box(-125.0011, 24.9493, -66.9326, 49.5904)
joined_cont = joined[joined.intersects(continental)]

In [ ]:
axes=joined_cont.plot(column='K202511_001E', figsize=(15,15))
axes.figure.dpi=190
axes

In [ ]:
joined_cont.head()

In [ ]:
import matplotlib as mpl
import numpy as np
inc_min=joined_cont['K202511_001E'].min()
inc_max=joined_cont['K202511_001E'].max()
X = [[0, 0], [.9, .9]]
plt.gca().imshow(X, interpolation='bicubic', cmap=mpl.cm.get_cmap(),
          extent=(0, .1, 0, 1), alpha=1)